---
title: Redfield Issue check "Analytically"
date: 2024-08-08
authors:
  - name: Gerardo Suarez
---

Since there seems to be an issue with the Bloch-Redfield Solver in qutip and my not so good implementation 
of the time dependent redfield, here's a quick Analytical check to make sure it's not, by solving the
equation for the SYK(2) model symbolically. The Hamiltonian is given by

In [1]:
from sympy import *
from sympy.physics.quantum import Dagger
from sympy import I
from collections import defaultdict
import numpy as np
import itertools
from sympy.solvers.ode.systems import dsolve_system

In [2]:
a = symbols('a',real=True,positive=True)

In [3]:
H = diag(-a,a,a,-a)#the times 20 is to rescale time
Eq(S('H'), H, evaluate=False)

Eq(H, Matrix([
[-a, 0, 0,  0],
[ 0, a, 0,  0],
[ 0, 0, a,  0],
[ 0, 0, 0, -a]]))

In [4]:
b= symbols('b0:8')

Here I define the coupling operator $q$ that make things break for Bloch-Redfield on the SYK model

In [5]:
# coupling operator non trivial like in the shinsey ryu paper. This difference is not dramatic for simpler couplings
Q = Matrix([[0, b[0] , b[1], 0],
              [conjugate(b[0]), 0, 0, -b[1]],
              [conjugate(b[1]), 0, 0, b[0]],
              [0, -conjugate(b[1]), conjugate(b[0]), 0]])

In [6]:
Eq(S("q"), Q, evaluate=False)

Eq(q, Matrix([
[            0,             b0,            b1,   0],
[conjugate(b0),              0,             0, -b1],
[conjugate(b1),              0,             0,  b0],
[            0, -conjugate(b1), conjugate(b0),   0]]))

In [7]:
Dagger(Q) == Q #checking I didn't mess up 

True

In [8]:
def comm(a,b):
    return (a*b-b*a)
def acomm(a,b):
    return (a*b+b*a)

I then get the eigenvalues and eigenvectors to obtain the jump operators (this steps are hidden in the pdf)

In [9]:
basis=H.eigenvects() # I probably don't need this step since the Hamiltonian is
# Diagonal
energies=[[i[0]]*i[1] for i in basis]
energies = [x for xs in energies for x in xs]
states = [i[2] for i in basis]
states = [x for xs in states for x in xs ]

In [10]:
def jump_operators(H,evals,all_state, Q):
    """
    A function to obtain the jump operators
    """
    collapse_list = []
    ws = []
    N=len(energies)
    for j in range(N):
        for k in range(j + 1, N):
            Deltajk = evals[k] - evals[j]
            ws.append(Deltajk)
            collapse_list.append(
                (
                    all_state[j]
                    * Dagger(all_state[j])
                    * Q
                    * all_state[k]
                    * Dagger(all_state[k])
                )
            )  # emission
            ws.append(-Deltajk)
            collapse_list.append(
                (
                    all_state[k]
                    * Dagger(all_state[k])
                    * Q
                    * all_state[j]
                    * Dagger(all_state[j])
                )
            )  # absorption
    collapse_list.append(Q - sum(collapse_list,zeros(N)))  # Dephasing
    ws.append(0)
    output = defaultdict(list)
    for k, key in enumerate(ws):
        output[key].append(collapse_list[k])
    eldict = {x: sum(y, zeros(N)) for x, y in output.items()}
    dictrem = {}
    empty = 0*H
    for keys, values in eldict.items():
        if not (values == empty):
            dictrem[keys] = values
    return dictrem

In [11]:
jumps=jump_operators(H,energies,states,Q)

In [12]:
for key,value in jumps.items():
    display(Eq(S(f"A({key})"), value, evaluate=False))

Eq(A(2*a), Matrix([
[0,             b0,            b1, 0],
[0,              0,             0, 0],
[0,              0,             0, 0],
[0, -conjugate(b1), conjugate(b0), 0]]))

Eq(A(-2*a), Matrix([
[            0, 0, 0,   0],
[conjugate(b0), 0, 0, -b1],
[conjugate(b1), 0, 0,  b0],
[            0, 0, 0,   0]]))

### Jump operator checks

The jump operators must satisfy

$$[H,A(\omega)]=-\omega A(\omega)$$
$$[H,A^{\dagger}(\omega) A(\omega)]=0$$
$$\sum_{w} A(\omega)= A$$


In [13]:
comm(H,jumps[2*a])/(-2*a) == jumps[2*a] #checking jump decomposition is fine

True

In [14]:
comm(H,Dagger(jumps[2*a])*jumps[2*a]) == zeros(4)

True

In [15]:
jumps[2*a]+jumps[-2*a]==Q

True

### Constucting the Differential equations
We now construct the differential equations from the GKLS form of the bloch 
Redfield generator

$$\begin{aligned}
     \dot{\rho_{S}^{I}(t)}{t} &=
    \sum_{\omega,\omega',\alpha,\beta} \gamma_{\beta,\alpha}(\omega,\omega')  \left( S_{\alpha}(\omega')  \rho^{I}_{S}(t) S^{\dagger}_{\beta}(\omega)- \frac{ \{S^{\dagger}_{\beta}(\omega) S_{\alpha}(\omega'),\rho^{I}_{S}(t)\}}{2} \right) \nonumber \\
    &+i \sum_{\omega,\omega',\alpha,\beta} S_{\beta,\alpha}(\omega,\omega') \Big[ \rho^{I}_{S}(t), S^{\dagger}_{\beta}(\omega) S_{\alpha}(\omega')\Big]\end{aligned}$$

I solve in the interaction picture generally, I did the same in my numerics so it should not be an issue (I rotate in the end).
By neglecting Lambshift as in the numerics

$$\dot{\rho_{S}^{I}(t)}{t} =
    \sum_{\omega,\omega',\alpha,\beta} \gamma_{\beta,\alpha}(\omega,\omega')  \left( S_{\alpha}(\omega')  \rho^{I}_{S}(t) S^{\dagger}_{\beta}(\omega)- \frac{ \{S^{\dagger}_{\beta}(\omega) S_{\alpha}(\omega'),\rho^{I}_{S}(t)\}}{2} \right)$$


In [16]:
# Simply define the symbolic density operator
rho = symbols('rho_1:17')
rho = Matrix(
    [rho[0: 4],
     rho[4: 8],
     rho[8: 12],
     rho[12:]]).subs(
    rho[-1],
    1 - rho[0] - rho[5] - rho[10])
rho=rho.subs({rho[4]: conjugate(rho[1]), rho[8]: conjugate(rho[2]),
          rho[12]:conjugate(rho[3]),rho[6]:conjugate(rho[9]),
          rho[13]: conjugate(rho[7]), rho[14]:conjugate(rho[11])})
Eq(S('rho'), rho, evaluate=False)


Eq(rho, Matrix([
[           rho_1,            rho_2,             rho_3,                       rho_4],
[conjugate(rho_2),            rho_6, conjugate(rho_10),                       rho_8],
[conjugate(rho_3),           rho_10,            rho_11,                      rho_12],
[conjugate(rho_4), conjugate(rho_8), conjugate(rho_12), -rho_1 - rho_11 - rho_6 + 1]]))

By hand I only need (2a,2a),(-2a,2a),(2a,-2a)

In [38]:
Lww=jumps[2*a]*rho *Dagger(jumps[2*a]) - acomm(Dagger(jumps[2*a])*jumps[2*a],rho)/2
Lww=simplify(Lww.expand().subs(b[0]*conjugate(b[0]),abs(b[0])**2).subs(b[1]*conjugate(b[1]),abs(b[1])**2))
Lww

Matrix([
[                                  b0*conjugate(b1)*conjugate(rho_10) + b1*rho_10*conjugate(b0) + rho_11*Abs(b1)**2 + rho_6*Abs(b0)**2,            rho_2*(-Abs(b0)**2 - Abs(b1)**2)/2,             rho_3*(-Abs(b0)**2 - Abs(b1)**2)/2,                                  b0**2*conjugate(rho_10) + b0*b1*rho_11 - b0*b1*rho_6 - b1**2*rho_10],
[                                                                                        (-Abs(b0)**2 - Abs(b1)**2)*conjugate(rho_2)/2,              rho_6*(-Abs(b0)**2 - Abs(b1)**2),   (-Abs(b0)**2 - Abs(b1)**2)*conjugate(rho_10),                                                                   rho_8*(-Abs(b0)**2 - Abs(b1)**2)/2],
[                                                                                        (-Abs(b0)**2 - Abs(b1)**2)*conjugate(rho_3)/2,             rho_10*(-Abs(b0)**2 - Abs(b1)**2),              rho_11*(-Abs(b0)**2 - Abs(b1)**2),                                                                  rho_12*(-Abs(b0)**2 - Abs(b

In [39]:
Lw1w=jumps[-2*a]*rho *Dagger(jumps[2*a]) - acomm(Dagger(jumps[2*a])*jumps[-2*a],rho)/2
Lw1w=simplify(Lww.expand().subs(b[0]*conjugate(b[0]),abs(b[0])**2).subs(b[1]*conjugate(b[1]),abs(b[1])**2))
Lw1w

Matrix([
[                                  b0*conjugate(b1)*conjugate(rho_10) + b1*rho_10*conjugate(b0) + rho_11*Abs(b1)**2 + rho_6*Abs(b0)**2,            rho_2*(-Abs(b0)**2 - Abs(b1)**2)/2,             rho_3*(-Abs(b0)**2 - Abs(b1)**2)/2,                                  b0**2*conjugate(rho_10) + b0*b1*rho_11 - b0*b1*rho_6 - b1**2*rho_10],
[                                                                                        (-Abs(b0)**2 - Abs(b1)**2)*conjugate(rho_2)/2,              rho_6*(-Abs(b0)**2 - Abs(b1)**2),   (-Abs(b0)**2 - Abs(b1)**2)*conjugate(rho_10),                                                                   rho_8*(-Abs(b0)**2 - Abs(b1)**2)/2],
[                                                                                        (-Abs(b0)**2 - Abs(b1)**2)*conjugate(rho_3)/2,             rho_10*(-Abs(b0)**2 - Abs(b1)**2),              rho_11*(-Abs(b0)**2 - Abs(b1)**2),                                                                  rho_12*(-Abs(b0)**2 - Abs(b

In [40]:
Lww1=jumps[-2*a]*rho *Dagger(jumps[2*a]) - acomm(Dagger(jumps[2*a])*jumps[-2*a],rho)/2
Lww1=simplify(Lww.expand().subs(b[0]*conjugate(b[0]),abs(b[0])**2).subs(b[1]*conjugate(b[1]),abs(b[1])**2))
Lww1

Matrix([
[                                  b0*conjugate(b1)*conjugate(rho_10) + b1*rho_10*conjugate(b0) + rho_11*Abs(b1)**2 + rho_6*Abs(b0)**2,            rho_2*(-Abs(b0)**2 - Abs(b1)**2)/2,             rho_3*(-Abs(b0)**2 - Abs(b1)**2)/2,                                  b0**2*conjugate(rho_10) + b0*b1*rho_11 - b0*b1*rho_6 - b1**2*rho_10],
[                                                                                        (-Abs(b0)**2 - Abs(b1)**2)*conjugate(rho_2)/2,              rho_6*(-Abs(b0)**2 - Abs(b1)**2),   (-Abs(b0)**2 - Abs(b1)**2)*conjugate(rho_10),                                                                   rho_8*(-Abs(b0)**2 - Abs(b1)**2)/2],
[                                                                                        (-Abs(b0)**2 - Abs(b1)**2)*conjugate(rho_3)/2,             rho_10*(-Abs(b0)**2 - Abs(b1)**2),              rho_11*(-Abs(b0)**2 - Abs(b1)**2),                                                                  rho_12*(-Abs(b0)**2 - Abs(b

In [41]:
generator=b[3]*simplify(2*Lww+Lww1+Lw1w) + I*comm(H,rho)
generator

Matrix([
[                                  b3*(4*b0*conjugate(b1)*conjugate(rho_10) + 4*b1*rho_10*conjugate(b0) + 4*rho_11*Abs(b1)**2 + 4*rho_6*Abs(b0)**2),                       -2*I*a*rho_2 - 2*b3*rho_2*(Abs(b0)**2 + Abs(b1)**2),                         -2*I*a*rho_3 - 2*b3*rho_3*(Abs(b0)**2 + Abs(b1)**2),                                  b3*(4*b0**2*conjugate(rho_10) + 4*b0*b1*rho_11 - 4*b0*b1*rho_6 - 4*b1**2*rho_10)],
[                                                                          2*I*a*conjugate(rho_2) - 2*b3*(Abs(b0)**2 + Abs(b1)**2)*conjugate(rho_2),                                     -4*b3*rho_6*(Abs(b0)**2 + Abs(b1)**2),                           -4*b3*(Abs(b0)**2 + Abs(b1)**2)*conjugate(rho_10),                                                                2*I*a*rho_8 - 2*b3*rho_8*(Abs(b0)**2 + Abs(b1)**2)],
[                                                                          2*I*a*conjugate(rho_3) - 2*b3*(Abs(b0)**2 + Abs(b1)**2)*conjugate(rho_3),         

In [42]:
mgene = Matrix(list(generator))
rhom=Matrix(list(rho))

In [43]:
t=Symbol('t',reals=True)

In [34]:
rhof = symbols('rho_1:17',cls=Function) # This is necessary to solve the differential equation
to_funcs={i:rhof[k](t) for k,i in enumerate(rhom)} #could have done so from the beginning though

In [35]:
mgene=mgene.subs(to_funcs) #substitute symbols for a function
mrhof=rhom.subs(to_funcs)

In [36]:
eqs = [Eq(mrhof[k].diff(t), mgene[k].simplify()) for k in range(16)]

In [37]:
for i in eqs:
    display(i)

Eq(Derivative(rho_1(t), t), -4*b3*(Abs(b0)**2 + Abs(b1)**2)*rho_1(t))

Eq(Derivative(rho_2(t), t), 2*(-I*a - b3*(Abs(b0)**2 + Abs(b1)**2))*rho_2(t))

Eq(Derivative(rho_3(t), t), 2*(-I*a - b3*(Abs(b0)**2 + Abs(b1)**2))*rho_3(t))

Eq(Derivative(rho_4(t), t), -4*b3*(Abs(b0)**2 + Abs(b1)**2)*rho_4(t))

Eq(Derivative(rho_5(t), t), 2*(I*a - b3*(Abs(b0)**2 + Abs(b1)**2))*rho_5(t))

...

Eq(Derivative(rho_12(t), t), 2*(I*a - b3*(Abs(b0)**2 + Abs(b1)**2))*rho_12(t))

Eq(Derivative(rho_13(t), t), -4*b3*(Abs(b0)**2 + Abs(b1)**2)*rho_13(t))

Eq(Derivative(rho_14(t), t), 2*(-I*a - b3*(Abs(b0)**2 + Abs(b1)**2))*rho_14(t))

Eq(Derivative(rho_15(t), t), 2*(-I*a - b3*(Abs(b0)**2 + Abs(b1)**2))*rho_15(t))

Eq(Derivative(rho_16(t), t), 4*b3*(rho_1(t)*Abs(b0)**2 + rho_1(t)*Abs(b1)**2 + rho_11(t)*Abs(b0)**2 + rho_11(t)*Abs(b1)**2 + rho_6(t)*Abs(b0)**2 + rho_6(t)*Abs(b1)**2 - Abs(b0)**2 - Abs(b1)**2))

Too many equations and symbols some of them are trivially cero for my initial conditions

In [34]:
zeros={rhof[1](t):0,rhof[2](t):0,rhof[4](t):0,rhof[7](t):0,rhof[8](t):0,rhof[11](t):0,rhof[13](t):0,rhof[14](t):0}

In [39]:
nontriv=[]
for i in eqs:
    if i.subs(zeros).rhs==0:
        pass
    else:
        display(i.subs(zeros))
        nontriv.append(i.subs(zeros))

Eq(Derivative(rho_1(t), t), 4*b3*(b0*rho_7(t)*conjugate(b1) + b1*rho_10(t)*conjugate(b0) + rho_11(t)*Abs(b1)**2 + rho_6(t)*Abs(b0)**2))

Eq(Derivative(rho_4(t), t), 4*b3*(b0**2*rho_7(t) + b0*b1*rho_11(t) - b0*b1*rho_6(t) - b1**2*rho_10(t)))

Eq(Derivative(rho_6(t), t), -4*b3*(Abs(b0)**2 + Abs(b1)**2)*rho_6(t))

Eq(Derivative(rho_7(t), t), -4*b3*(Abs(b0)**2 + Abs(b1)**2)*rho_7(t))

Eq(Derivative(rho_10(t), t), -4*b3*(Abs(b0)**2 + Abs(b1)**2)*rho_10(t))

Eq(Derivative(rho_11(t), t), -4*b3*(Abs(b0)**2 + Abs(b1)**2)*rho_11(t))

Eq(Derivative(rho_13(t), t), 4*b3*(rho_10(t)*conjugate(b0)**2 + rho_11(t)*conjugate(b0)*conjugate(b1) - rho_6(t)*conjugate(b0)*conjugate(b1) - rho_7(t)*conjugate(b1)**2))

Eq(Derivative(rho_16(t), t), 4*b3*(-b0*rho_7(t)*conjugate(b1) - b1*rho_10(t)*conjugate(b0) + rho_11(t)*Abs(b0)**2 + rho_6(t)*Abs(b1)**2))

In [ ]:
sols = dsolve(
    nontriv,
    ics={rhof[0] (0): 0, rhof[3] (0): 0,
        rhof[5] (0): 0.5, rhof[6] (0): 0.5 * I
        , rhof[9] (0): -0.5 * I, rhof[10] (0): 0.5,
        rhof[12](0): 0,
         rhof[15](0): 0})

In [ ]:
for i in sols:
    display(i)

In [ ]:
old={**chgnvar,**chgnvar2}

In [ ]:
inverted = {value: key for key, value in old.items()}

In [ ]:
newsols=[i.subs(inverted).simplify() for i in sols]

By reverting the change of variables. The analytical answer of the Bloch redfield equation is given by

In [ ]:
ans=Matrix(np.array([i.rhs for i in newsols]).reshape(4,4))

In [ ]:
for i in newsols:
    display(i)

We can then substitute the numerical values for example for the case we explored above

In [ ]:
num_values = {w0: 4.799892154530513, lam: 2.157529680721121, gam: 2.3414108070880553, a: 2.182,b[0]:0.53,b[1]:0.707,b[2]:0.442,b[3]:0.471}

In [ ]:
anss=ans.subs(num_values).evalf().expand()

In [ ]:
Eq(S("rho(t)"),anss,evaluate=False)

Then we may evaluate for long times

In [ ]:
def roundMatrix(m, n): 
    return Matrix(
    [[round(m[x, y], n) for y in range(m.shape[1])] for x in range(m.shape[0])])

In [ ]:
Eq(S("rho(150)"),roundMatrix(ans.subs(num_values).subs(t,150).evalf(),18),evaluate=False)

I believe I was careful enough to use the same convention used in the other equations (Pseudomodes, Cumulant and redfield ) but currently reviewing the derivations to make sure there's no inconsistencies. The derivations in question are in https://master--gsuarezthesis.netlify.app/redfield . I do think it is now safe to assume that BR/Redfield breaksdown and that it is not a bug in the code, so maybe we can write a paper on redfield breaking down, cumulant/global being good once we figure out why it happens🤓. Though it seems to be about the coupling and not the degeneracies

:::{tip} About Pictures
Technically the above matrix is not correct as it is in the interaction picture and not the Schrodinger picture. In this case it does not make a difference, however, let us do the rotation 
:::

In [ ]:
U=exp(I*H*t)

In [ ]:
Eq(S("U"),U,evaluate=False)


In [ ]:

Eq(S("U")*S("rho")*Dagger(S("U")),U*rho*Dagger(U),evaluate=False)


In [ ]:
rhoss=U*roundMatrix(ans.subs(num_values).subs(t,150).evalf(),18)*Dagger(U)

In [ ]:
Eq(S("rho(150)"),rhoss,evaluate=False)


## RC picture of the Hamiltonian

For the RC I simply follow https://arxiv.org/pdf/1511.05181

So If I didn't misunderstand it then

$$\Omega=w_{0}$$
$$\lambda_{rc}=\sqrt{\frac{\pi}{2 w_{0}}}\lambda$$


Not actually sure if $\pi$ should be there, but does not seem to be relevant for the question we are asking

In [ ]:
def ladder(i,j):
    if i - j == 1:
        return sqrt(i)
    else:
        return 0
def destroy(n):
    return Dagger(Matrix(n,n,ladder))


Not sure how many levels to take here, but let us guess 15 is enough

In [ ]:
levels=15
arc=destroy(levels)
arcd=Dagger(arc)

Then I construct the RC Hamiltonian

In [ ]:
from sympy.physics.quantum import TensorProduct

In [ ]:
HRC = TensorProduct(H, eye(levels))+sqrt(pi/(2*w0))*lam*TensorProduct(Q,eye(levels))*(TensorProduct(eye(4),arc)+TensorProduct(eye(4),arcd))+w0*TensorProduct(eye(4),arcd)*TensorProduct(eye(4),arc)

In [ ]:
NHRC=HRC.subs(num_values).evalf()

In [ ]:
ans=np.array(NHRC.tolist()).astype(np.complex128)

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(ans)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(np.real(eigenvalues),np.round(np.imag(eigenvalues),10),s=5)
plt.title("Eigenvalues on the real line")
plt.show()

Propably I shopuld have done the partial trace so

In [ ]:
import qutip as qt

In [ ]:
Hq=qt.Qobj(H.subs(num_values).tolist())
aq=qt.destroy(levels)
aqd=qt.create(levels)
qHRC = qt.tensor(Hq, qt.qeye(levels))+np.sqrt(np.pi/(2*num_values[w0]))*num_values[lam]*qt.tensor(qt.Qobj(Q.subs(num_values)), qt.qeye(levels))*(qt.tensor(qt.qeye(4),aq)+qt.tensor(qt.qeye(4),aqd))+num_values[w0]*qt.tensor(qt.qeye(4),aqd)*qt.tensor(qt.qeye(4),aq)

In [ ]:
Hq

In [ ]:
qHRC.ptrace(0)

Still degenerate

Let us try the other coupling which does not break bloch redfield

In [ ]:
Hq=qt.Qobj(H.subs(num_values).tolist())
aq=qt.destroy(levels)
aqd=qt.create(levels)
qHRC = qt.tensor(Hq, qt.qeye(levels))+np.sqrt(np.pi/(2*num_values[w0]))*num_values[lam]*qt.tensor(qt.Qobj(re(Q).subs(num_values)), qt.qeye(levels))*(qt.tensor(qt.qeye(4),aq)+qt.tensor(qt.qeye(4),aqd))+num_values[w0]*qt.tensor(qt.qeye(4),aqd)*qt.tensor(qt.qeye(4),aq)

In [ ]:
qHRC.ptrace(0)

In [ ]:
ans = qHRC.full().astype(np.complex128)

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(ans)

In [ ]:
plt.scatter(np.real(eigenvalues), np.round(np.imag(eigenvalues), 10), s=5)
plt.title("Eigenvalues on the real line")
plt.show()

There doesn't seem to be much change in the Hamiltonian if any